# 질의 응답

In [8]:
import os
print(os.getcwd())

c:\Users\Admin\openai\openai-api-tutorial\test\info


In [9]:
import pandas as pd
import re
from openai import OpenAI

# CSV 파일 불러오기
full_df = pd.read_csv('C:/Users/Admin/openai/openai-api-tutorial/test/info/전체관광지_최종분류.csv')

# 지역/계절/주제 리스트 준비
regions = full_df['광역지역'].dropna().unique().tolist()
seasons = ["봄", "여름", "가을", "겨울"]
topics = ["문화유산", "축제", "특산물", "관광지", "숙박"]

In [10]:
# 키워드 추출
def extract_keywords(user_input):
    region = next((r for r in regions if r.replace(" ", "") in user_input.replace(" ", "")), None)
    season = next((s for s in seasons if s in user_input), None)
    topic = next((t for t in topics if t in user_input), None)
    return region, season, topic

In [11]:
# DB 조회
def query_database(region=None, season=None, topic=None):
    query = full_df.copy()
    if region:
        query = query[query['광역지역'].str.contains(region, na=False)]
    if topic:
        if topic == "숙박":
            query = query[query['분류'] == "Accommodation"]
        elif topic == "축제":
            query = query[query['분류'] == "Festival"]
        elif topic == "관광지":
            query = query[query['분류'] == "Place"]
        elif topic == "문화유산":
            query = query[query['분류'] == "Leisure"]
    # 시즌 필터링은 추후 확장
    return query

In [16]:
# LLM 응답 생성
def generate_response(user_input, db_result):
    client = OpenAI(api_key="sk-proj-HAE_mROBCHMh6rVXT8zVpJ7T-" \
"oWy_owEptIF1JKugQYn_02_V5P_h43G1L7pgPAkMf4A-" \
"_ktCXT3BlbkFJsRfIiCGFIRCnBBvs_g2nYXI6hS1y88Xo7uvWsvxffvkkHe8wkfFPb-6BPMupmou9v0syllP-kA")
    prompt = f"""
    사용자의 요청: {user_input}
    데이터베이스 검색 결과 (참고용):\n{db_result[['광역지역','세부지역','관광지명','설명']].head(3).to_string(index=False)}
    
    위 정보를 참고하여 사용자 요청에 맞게 최적화된 답변을 생성해주세요.
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content


In [18]:
# 검토 리포트 생성
def generate_report(region, season, topic, db_result):
    return {
        "사용자 입력 키워드": {"지역": region, "계절": season, "주제": topic},
        "DB 검색 결과 유무": "있음" if not db_result.empty else "없음",
        "DB 결과 수": len(db_result),
    }

In [19]:
# 전체 연결
def full_process(user_input):
    # 1. 키워드 추출
    region, season, topic = extract_keywords(user_input)
    # 2. DB 검색
    db_result = query_database(region=region, season=season, topic=topic)
    # 3. LLM 응답 생성
    final_response = generate_response(user_input, db_result)
    # 4. 검토 리포트 생성
    report = generate_report(region, season, topic, db_result)
    return final_response, report


In [34]:
# 계절 필터링 로직 추가
def season_filter(query, season):
    if season:
        season_keywords = {
            "봄": ["봄", "3월", "4월", "5월"],
            "여름": ["여름", "6월", "7월", "8월"],
            "가을": ["가을", "9월", "10월", "11월"],
            "겨울": ["겨울", "12월", "1월", "2월"]
        }
        keywords = season_keywords.get(season, [])
        pattern = '|'.join(keywords)
        # '설명'이나 '관광지명' 안에 계절 관련 키워드가 있으면 필터링
        query = query[query['설명'].str.contains(pattern, na=False) | query['관광지명'].str.contains(pattern, na=False)]
    return query

In [35]:
# 질문 인식 로직 추가
def clean_user_input(user_input):
    # 다양한 요청 표현을 미리 정의
    request_phrases = ["추천해줘", "소개해줘", "알려줘", "추천 받을 수 있을까", "추천 부탁해", "추천해", "추천"]
    for phrase in request_phrases:
        user_input = user_input.replace(phrase, "")
    return user_input.strip()

In [36]:
def full_process(user_input):
    # 0. 사용자 입력 정리
    cleaned_input = clean_user_input(user_input)
    # 1. 키워드 추출
    region, season, topic = extract_keywords(cleaned_input)
    # 2. DB 검색
    db_result = query_database(region=region, topic=topic)
    db_result = season_filter(db_result, season)
    # 3. LLM 응답 생성
    final_response = generate_response(user_input, db_result)
    # 4. 검토 리포트 생성
    report = generate_report(region, season, topic, db_result)
    return final_response, report

In [42]:
# 먼저 매핑 테이블 추가
region_correction = {
    "서울": "서울특별시",
    "부산": "부산광역시",
    "대구": "대구광역시",
    "인천": "인천광역시",
    "광주": "광주광역시",
    "대전": "대전광역시",
    "울산": "울산광역시",
    "세종": "세종특별자치시",
    "경기": "경기도",
    "강원": "강원특별자치도",
    "충북": "충청북도",
    "충남": "충청남도",
    "전북": "전라북도",
    "전남": "전라남도",
    "경북": "경상북도",
    "경남": "경상남도",
    "제주": "제주특별자치도"
}

# 수정된 extract_keywords 함수
def extract_keywords(user_input):
    region = None
    for short_name, full_name in region_correction.items():
        if short_name in user_input:
            region = full_name
            break

    season = next((s for s in seasons if s in user_input), None)
    topic = next((t for t in topics if t in user_input), None)
    return region, season, topic

In [37]:
# 질문 공식
# (지역명)(에서/에) (계절)(에) 갈 수 있는 (주제) (추천해줘/알려줘/소개해줘)

In [43]:
# 1. 사용자 입력 준비
user_input = "서울에서 봄에 갈만한 관광지 추천해줘"  # ✅ 여기 질문 바꿔서 입력하면 됨

# 2. full_process 함수 호출
response, report = full_process(user_input)
# 3. 결과 출력
print("✅ 답변:\n", response)
print("\n📝 검토 리포트:\n", report)

✅ 답변:
 서울에서 봄에 방문하기 좋은 관광지를 추천해드리겠습니다.

1. **해아원**: 서울시 종로구에 위치한 전통 한옥 스테이로, '이슬 기운처럼 예쁜 집'이라는 뜻입니다. 이곳은 상위 1%가 누릴 수 있는 최고급 가구와 어메니티를 제공하여 편안하고 프라이빗한 경험을 선사합니다. 전통 한옥의 아름다움과 함께 잊지 못할 추억을 만들어 보세요.

2. **서울숲**: 다양한 식물과 동물, 그리고 예술 작품이 조화를 이루는 서울숲은 봄철에 특히 아름답습니다. 벚꽃이 만개하는 시기에는 산책로를 따라 걷기만 해도 봄의 정취를 만끽할 수 있습니다.

3. **남산공원**: 서울의 대표적인 명소 중 하나인 남산공원은 봄이 되면 벚꽃과 다양한 꽃들이 만개하여 아름다운 경관을 자랑합니다. 남산타워 전망대에서 서울의 전경을 감상하는 것도 추천드립니다.

4. **창덕궁 후원**: 유네스코 세계문화유산으로 지정된 창덕궁의 비밀정원인 후원은 봄철에 더욱 특별한 매력을 발산합니다. 사전 예약을 통해 한정된 인원만이 관람할 수 있어 조용한 시간을 보낼 수 있습니다.

이 외에도 서울에는 다양한 봄철 관광지가 많으니, 각자의 취향에 맞는 장소를 찾아보시는 것도 좋을 것 같습니다. 즐거운 여행 되세요!

📝 검토 리포트:
 {'사용자 입력 키워드': {'지역': '서울특별시', '계절': '봄', '주제': '관광지'}, 'DB 검색 결과 유무': '있음', 'DB 결과 수': 2}


In [44]:
user_input = "부산 여름 축제 추천해줘"
response, report = full_process(user_input)

print("✅ 답변:\n", response)
print("\n📝 검토 리포트:\n", report)

✅ 답변:
 부산에서 여름에 즐길 수 있는 축제를 추천해드리겠습니다. 비록 데이터베이스에 관련 정보가 없지만, 부산은 다양한 여름 축제로 유명합니다. 다음은 부산에서 열리는 대표적인 여름 축제들입니다:

1. **부산 바다축제**: 매년 8월 초 해운대, 광안리 등 부산의 주요 해변에서 열리는 축제로, 다양한 해양 스포츠 체험과 공연, 불꽃놀이 등이 진행됩니다.

2. **부산 국제 록 페스티벌**: 8월 초 삼락생태공원에서 열리는 이 페스티벌은 국내외 유명 록 밴드들이 참여하여 뜨거운 여름 밤을 즐길 수 있는 기회를 제공합니다.

3. **부산 국제 코미디 페스티벌**: 8월 말부터 9월 초까지 다양한 코미디 공연과 이벤트가 부산 전역에서 펼쳐지는 축제로, 웃음과 재미를 선사합니다.

4. **부산 국제 매직 페스티벌**: 8월 중순에 열리는 이 축제는 세계 각국의 마술사들이 참여하여 다양한 마술 공연을 선보입니다.

이 외에도 부산에서는 여름 동안 다양한 문화와 예술 관련 행사들이 진행되니, 방문 시기에 맞춰 즐길 수 있는 축제를 확인해보시기 바랍니다. 즐거운 부산 여름 여행 되세요!

📝 검토 리포트:
 {'사용자 입력 키워드': {'지역': '부산광역시', '계절': '여름', '주제': '축제'}, 'DB 검색 결과 유무': '없음', 'DB 결과 수': 0}


In [45]:
user_input = "경기도에서 겨울에 할 수 있는 문화유산 소개해줘"
response, report = full_process(user_input)

print("✅ 답변:\n", response)
print("\n📝 검토 리포트:\n", report)

✅ 답변:
 경기도는 다양한 문화유산과 함께 겨울철에도 즐길 수 있는 여러 관광 명소가 있는 지역입니다. 겨울에 경기도에서 방문할 만한 문화유산 몇 가지를 소개합니다:

1. **수원 화성**: 유네스코 세계문화유산으로 지정된 수원 화성은 겨울철에도 아름다운 경관을 자랑합니다. 눈이 내린 후의 화성은 특히 아름다우며, 성곽을 따라 산책하며 겨울의 운치를 느낄 수 있습니다.

2. **남한산성**: 남한산성도 유네스코 세계문화유산으로, 겨울철에 하이킹을 즐기기에 좋은 장소입니다. 눈 덮인 산성과 주변의 경치는 겨울의 고요함을 만끽하기에 적합합니다.

3. **양주 회암사지**: 양주에 위치한 회암사지는 고려시대의 사찰 유적으로, 조용한 산속에 있어 겨울철에 방문하기 좋습니다. 눈 덮인 사지의 풍경은 고즈넉한 분위기를 자아냅니다.

4. **파주 임진각**: 임진각은 한국 전쟁의 역사를 간직한 장소로, 겨울철에도 많은 방문객이 찾는 곳입니다. 주변의 평화누리공원에서는 겨울철 야외 조각 작품을 감상할 수 있습니다.

5. **용인 한국민속촌**: 겨울철에는 전통 겨울 축제가 열리며, 조선 시대의 겨울 생활을 체험할 수 있는 다양한 프로그램이 제공됩니다. 전통 놀이와 먹거리를 즐기며 한국의 겨울 문화를 경험할 수 있습니다.

각 장소는 겨울철에만 느낄 수 있는 독특한 매력을 제공합니다. 방한 준비를 잘 하시고 경기도의 문화유산을 탐방해보세요!

📝 검토 리포트:
 {'사용자 입력 키워드': {'지역': '경기도', '계절': '겨울', '주제': '문화유산'}, 'DB 검색 결과 유무': '없음', 'DB 결과 수': 0}


# 대한민국 문화 추천 받는 랭체인

In [61]:
# 필요한 라이브러리 불러오기
import pandas as pd
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 데이터베이스 CSV 파일 불러오기
full_df = pd.read_csv('C:/Users/Admin/openai/openai-api-tutorial/test/info/전체관광지_최종분류.csv')

# 7가지 카테고리 리스트
categories = {
    "Festivals": "축제",
    "Restaurants": "레스토랑",
    "Shopping": "쇼핑",
    "Accommodation": "숙박",
    "Leisure": "레저",
    "Cultural facilities": "문화시설",
    "Places": "장소"
}

# 지역, 계절, 주제 키워드 추출
regions = full_df['광역지역'].dropna().unique().tolist()
seasons = ["봄", "여름", "가을", "겨울"]
topics = list(categories.keys())

# OpenAI 모델 설정 (API 키 넣어야 함)
openai_api_key = "sk-proj-HAE_mROBCHMh6rVXT8zVpJ7T-" \
"oWy_owEptIF1JKugQYn_02_V5P_h43G1L7pgPAkMf4A-" \
"_ktCXT3BlbkFJsRfIiCGFIRCnBBvs_g2nYXI6hS1y88Xo7uvWsvxffvkkHe8wkfFPb-6BPMupmou9v0syllP-kA"
llm = OpenAI(api_key=openai_api_key)

# 템플릿 정의
prompt_template = """
사용자가 요청한 내용을 바탕으로 아래 조건에 맞는 추천을 해주세요:
- 지역: {region}
- 계절: {season}
- 주제: {category}

추천 항목을 목록으로 제시해 주세요.
"""

prompt = PromptTemplate(input_variables=["region", "season", "category"], template=prompt_template)
chain = LLMChain(llm=llm, prompt=prompt)

# 지역 축약어를 풀네임으로 매핑하는 딕셔너리
region_correction = {
    "서울": "서울특별시",
    "부산": "부산광역시",
    "대구": "대구광역시",
    "인천": "인천광역시",
    "광주": "광주광역시",
    "대전": "대전광역시",
    "울산": "울산광역시",
    "세종": "세종특별자치시",
    "경기": "경기도",
    "강원": "강원특별자치도",
    "충북": "충청북도",
    "충남": "충청남도",
    "전북": "전라북도",
    "전남": "전라남도",
    "경북": "경상북도",
    "경남": "경상남도",
    "제주": "제주특별자치도"
}

# DB에서 필터링된 결과를 LangChain에 전달하여 자연스럽게 추천 생성
def query_database(region=None, season=None, category=None):
    query = full_df.copy()
    
    # 지역 필터링
    if region:
        query = query[query['광역지역'].str.contains(region, na=False)]
    
    # 주제 필터링
    if category:
        query = query[query['분류'].str.contains(categories.get(category, ""), na=False)]
    
    # 계절 필터링 (계절은 설명 또는 관광지명에 해당 키워드가 있으면 필터링)
    if season:
        season_keywords = {
            "봄": ["봄", "3월", "4월", "5월"],
            "여름": ["여름", "6월", "7월", "8월"],
            "가을": ["가을", "9월", "10월", "11월"],
            "겨울": ["겨울", "12월", "1월", "2월"]
        }
        keywords = season_keywords.get(season, [])
        pattern = '|'.join(keywords)
        query = query[query['설명'].str.contains(pattern, na=False) | query['관광지명'].str.contains(pattern, na=False)]
    
    return query

# 사용자 입력에서 지역, 계절, 카테고리 추출
def extract_keywords(user_input):
    region = None
    for short_name, full_name in region_correction.items():
        if short_name in user_input:
            region = full_name
            break

    season = next((s for s in seasons if s in user_input), None)
    category = next((cat for cat in categories.keys() if cat.lower() in user_input.lower()), None)
    
    return region, season, category

# 추천 응답 생성
def generate_response(user_input, region, season, category):
    # DB에서 필터링된 결과
    db_result = query_database(region=region, season=season, category=category)
    
    # LangChain을 통해 추천 문장 생성
    response = chain.run(region=region, season=season, category=category)
    
    return response

# 전체 프로세스
def full_process(user_input):
    # 1. 사용자 질문에서 키워드 추출
    region, season, category = extract_keywords(user_input)
    
    # 2. 모델을 통해 추천 답변 생성
    response = generate_response(user_input, region, season, category)
    
    return response


In [62]:
# 예시 질문
user_input = "부산 여름 축제 추천해줘"
response = full_process(user_input)

print("✅ 답변:\n", response)

✅ 답변:
 
1. 해운대 해수욕장 방문
2. 부산 국제영화제 관람
3. 동백섬 산책
4. 광안리 해변에서 수영 및 해변 축제 참여
5. 부산 태종대 방문
6. 송정해수욕장에서 물놀이
7. 영화의 전당에서 영화 관람
8. 남포동 거리에서 쇼핑 및 맛집 탐방
9. 해운대 달맞이길 산책
10. 부산 태종대 수족관 방문


# 최종

In [2]:
import pandas as pd
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 데이터베이스 CSV 파일 불러오기
full_df = pd.read_csv('C:/Users/Admin/openai/openai-api-tutorial/test/info/전체관광지_최종분류.csv')

# 7가지 카테고리 리스트
categories = {
    "Festivals": "축제",
    "Restaurants": "레스토랑",
    "Shopping": "쇼핑",
    "Accommodation": "숙박",
    "Leisure": "레저",
    "Cultural facilities": "문화시설",
    "Places": "장소"
}

# 지역, 계절, 주제 키워드 추출
regions = full_df['광역지역'].dropna().unique().tolist()
seasons = ["봄", "여름", "가을", "겨울"]
topics = list(categories.keys())

# OpenAI 모델 설정 (API 키 넣어야 함)
openai_api_key = "sk-proj-HAE_mROBCHMh6rVXT8zVpJ7T-" \
"oWy_owEptIF1JKugQYn_02_V5P_h43G1L7pgPAkMf4A-" \
"_ktCXT3BlbkFJsRfIiCGFIRCnBBvs_g2nYXI6hS1y88Xo7uvWsvxffvkkHe8wkfFPb-6BPMupmou9v0syllP-kA"
llm = OpenAI(api_key=openai_api_key)

# 템플릿 정의
prompt_template = """
사용자가 요청한 내용을 바탕으로 아래 조건에 맞는 추천을 해주세요:
- 지역: {region}
- 계절: {season}
- 주제: {category}

추천 항목을 목록으로 제시해 주세요.
"""

prompt = PromptTemplate(input_variables=["region", "season", "category"], template=prompt_template)
chain = LLMChain(llm=llm, prompt=prompt)

# 지역 축약어를 풀네임으로 매핑하는 딕셔너리
region_correction = {
    "서울": "서울특별시",
    "부산": "부산광역시",
    "대구": "대구광역시",
    "인천": "인천광역시",
    "광주": "광주광역시",
    "대전": "대전광역시",
    "울산": "울산광역시",
    "세종": "세종특별자치시",
    "경기": "경기도",
    "강원": "강원특별자치도",
    "충북": "충청북도",
    "충남": "충청남도",
    "전북": "전라북도",
    "전남": "전라남도",
    "경북": "경상북도",
    "경남": "경상남도",
    "제주": "제주특별자치도"
}

# DB에서 필터링된 결과를 LangChain에 전달하여 자연스럽게 추천 생성
def query_database(region=None, season=None, category=None):
    query = full_df.copy()
    
    # 지역 필터링
    if region:
        query = query[query['광역지역'].str.contains(region, na=False)]
    
    # 주제 필터링
    if category:
        query = query[query['분류'].str.contains(categories.get(category, ""), na=False)]
    
    # 계절 필터링 (계절은 설명 또는 관광지명에 해당 키워드가 있으면 필터링)
    if season:
        season_keywords = {
            "봄": ["봄", "3월", "4월", "5월"],
            "여름": ["여름", "6월", "7월", "8월"],
            "가을": ["가을", "9월", "10월", "11월"],
            "겨울": ["겨울", "12월", "1월", "2월"]
        }
        keywords = season_keywords.get(season, [])
        pattern = '|'.join(keywords)
        query = query[query['설명'].str.contains(pattern, na=False) | query['관광지명'].str.contains(pattern, na=False)]
    
    return query

# 사용자 입력에서 지역, 계절, 카테고리 추출
def extract_keywords(user_input):
    region = None
    for short_name, full_name in region_correction.items():
        if short_name in user_input:
            region = full_name
            break

    season = next((s for s in seasons if s in user_input), None)
    
    # 활동 추출
    activity = next((cat for cat in categories.keys() if cat.lower() in user_input.lower()), None)

    return region, season, activity

# 추천 응답 생성
def generate_response(user_input, region, season, activity):
    # DB에서 필터링된 결과
    db_result = query_database(region=region, season=season, category=activity)
    
    # LangChain을 통해 추천 문장 생성
    response = chain.run(region=region, season=season, category=activity)
    
    return response

# 전체 프로세스
def full_process(user_input):
    # 1. 사용자 질문에서 키워드 추출
    region, season, activity = extract_keywords(user_input)
    
    # 2. 모델을 통해 추천 답변 생성
    response = generate_response(user_input, region, season, activity)
    
    return response


C:\Users\Admin\AppData\Local\Temp\ipykernel_15100\1973301229.py:29: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(api_key=openai_api_key)
C:\Users\Admin\AppData\Local\Temp\ipykernel_15100\1973301229.py:42: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [3]:
# 예시 질문
user_input = "강릉 가을 축제 추천해줘"
response = full_process(user_input)

print("✅ 답변:\n", response)

✅ 답변:
 
- fall foliage 관광
- 가을 테마 파크/축제
- 산책/하이킹 코스
- 가을 풍경을 담은 사진관광
- 가을 맛집 투어
- 포토존 투어 (가을 빛깔이 아름다운 장소)
- 가을 풍경을 감상할 수 있는 카페/레스토랑 방문
- 가을 단풍 구경을 위한 드라이브 코스
- 가을 농장 체험 (과일 따기, 옥수수 도시락 만들기 등)
- 가을 맞이 축제 (단풍축제, 가을 먹거리 축제 등)


In [ ]:
# 수정
import pandas as pd
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 데이터베이스 CSV 파일 불러오기
full_df = pd.read_csv('C:/Users/Admin/openai/openai-api-tutorial/test/info/전체관광지_최종분류.csv')

# 7가지 카테고리 리스트
categories = {
    "Festivals": "축제",
    "Restaurants": "레스토랑",
    "Shopping": "쇼핑",
    "Accommodation": "숙박",
    "Leisure": "레저",
    "Cultural facilities": "문화시설",
    "Places": "장소"
}

# 지역, 계절, 주제 키워드 추출
regions = full_df['광역지역'].dropna().unique().tolist()
seasons = ["봄", "여름", "가을", "겨울"]
topics = list(categories.keys())

# OpenAI 모델 설정 (API 키 넣어야 함)
openai_api_key = "sk-proj-HAE_mROBCHMh6rVXT8zVpJ7T-" \
"oWy_owEptIF1JKugQYn_02_V5P_h43G1L7pgPAkMf4A-" \
"_ktCXT3BlbkFJsRfIiCGFIRCnBBvs_g2nYXI6hS1y88Xo7uvWsvxffvkkHe8wkfFPb-6BPMupmou9v0syllP-kA"
llm = OpenAI(api_key=openai_api_key)

# 템플릿 정의
prompt_template = """
사용자가 요청한 내용을 바탕으로 아래 조건에 맞는 추천을 해주세요:
- 지역: {region}
- 계절: {season}
- 주제: {category}

추천 항목을 목록으로 제시해 주세요.
"""

prompt = PromptTemplate(input_variables=["region", "season", "category"], template=prompt_template)
chain = LLMChain(llm=llm, prompt=prompt)

# 지역 축약어를 풀네임으로 매핑하는 딕셔너리
region_correction = {
    "서울": "서울특별시",
    "부산": "부산광역시",
    "대구": "대구광역시",
    "인천": "인천광역시",
    "광주": "광주광역시",
    "대전": "대전광역시",
    "울산": "울산광역시",
    "세종": "세종특별자치시",
    "경기": "경기도",
    "강원": "강원특별자치도",
    "충북": "충청북도",
    "충남": "충청남도",
    "전북": "전라북도",
    "전남": "전라남도",
    "경북": "경상북도",
    "경남": "경상남도",
    "제주": "제주특별자치도"
}

# DB에서 필터링된 결과를 LangChain에 전달하여 자연스럽게 추천 생성
def query_database(region=None, season=None, category=None):
    query = full_df.copy()
    
    # 지역 필터링
    if region:
        query = query[query['광역지역'].str.contains(region, na=False)]
    
    # 주제 필터링 (7가지 카테고리로 필터링)
    if category:
        category_map = {
            "Festivals": "축제",
            "Restaurants": "레스토랑",
            "Shopping": "쇼핑",
            "Accommodation": "숙박",
            "Leisure": "레저",
            "Cultural facilities": "문화시설",
            "Places": "장소"
        }
        query = query[query['분류'].str.contains(category_map.get(category, ""), na=False)]
    
    # 계절 필터링 (계절은 설명 또는 관광지명에 해당 키워드가 있으면 필터링)
    if season:
        season_keywords = {
            "봄": ["봄", "3월", "4월", "5월"],
            "여름": ["여름", "6월", "7월", "8월"],
            "가을": ["가을", "9월", "10월", "11월"],
            "겨울": ["겨울", "12월", "1월", "2월"]
        }
        keywords = season_keywords.get(season, [])
        pattern = '|'.join(keywords)
        query = query[query['설명'].str.contains(pattern, na=False) | query['관광지명'].str.contains(pattern, na=False)]
    
    return query

# 사용자 입력에서 지역, 계절, 카테고리 추출
def extract_keywords(user_input):
    region = None
    for short_name, full_name in region_correction.items():
        if short_name in user_input:
            region = full_name
            break

    season = next((s for s in seasons if s in user_input), None)
    
    # 활동 추출
    activity = next((cat for cat in categories.keys() if cat.lower() in user_input.lower()), None)

    return region, season, activity

# 추천 응답 생성
def generate_response(user_input, region, season, activity):
    # DB에서 필터링된 결과
    db_result = query_database(region=region, season=season, category=activity)
    
    # LangChain을 통해 추천 문장 생성
    response = chain.run(region=region, season=season, category=activity)
    
    return response

# 전체 프로세스
def full_process(user_input):
    # 1. 사용자 질문에서 키워드 추출
    region, season, activity = extract_keywords(user_input)
    
    # 2. 모델을 통해 추천 답변 생성
    response = generate_response(user_input, region, season, activity)
    
    return response


In [7]:
# 예시 질문
user_input = "부산 여름 숙박 추천해줘"
response = full_process(user_input)

print("✅ 답변:\n", response)

✅ 답변:
 
1. 해운대 해수욕장 방문
2. 광안리 해변에서 물놀이 즐기기
3. 동백섬 산책로 걷기
4. 태종대에서 바다 전망 즐기기
5. 해운대 마린시티에서 수상스포츠 체험하기
6. 오륙도 해안도로 드라이브 즐기기
7. 국제시장에서 부산의 다양한 음식 즐기기
8. 영도 대교에서 부산 야경 감상하기
9. 송정해수욕장에서 캠핑 즐기기
10. 부산 타워에서 전망대에서 부산 시내 전망 즐기기
